In [47]:
from plotly import express as px

In [48]:
import plotly.graph_objects as go
import polars as pl
import torch

from cci.utils import project_dir

# res_dir = project_dir() / "results/CNN/32"
# res_dir = project_dir() / "results/CNN/40"
# res_dir = project_dir() / "results/CNN/47"
# res_dir = project_dir() / "results/CNN/65"
# res_dir = project_dir() / "results/CNN/101"
# res_dir = project_dir() / "results/CNN/114"
# Her++++++++++++++++++++++++++++++++++++++++++++++++4444
# res_dir = project_dir() / "results/CNN/115"4
# res_dir = project_dir() / "results/CNN/136"
# res_dir = project_dir() / "results/CNN/153"
# res_dir = project_dir() / "results/CNN/161"
# res_dir = project_dir() / "results/CNN/169"
# res_dir = project_dir() / "results/CNN/171"
# res_dir = project_dir() / "results/CNN/174"

res_dir = project_dir() / "results/CNN_vft/91"
# model = torch.load(res_dir / "0_model.pt", map_location=torch.device("cpu"))
# print()

In [49]:
test_df = pl.read_csv(res_dir / "*_test.csv").drop("epoch")
test_df

acc,loss,f1,auroc,precision,recall,specificity,bac,TP,FP,FN,TN
f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64
0.757009,0.544618,0.857143,0.585632,0.821053,0.896552,0.15,0.523276,3,17,9,78
0.738318,0.58615,0.842697,0.697701,0.824176,0.862069,0.2,0.531034,4,16,12,75
0.803738,0.471084,0.888889,0.683908,0.823529,0.965517,0.1,0.532759,2,18,3,84
0.71028,0.542414,0.818713,0.665517,0.833333,0.804598,0.3,0.552299,6,14,17,70
0.71028,0.543705,0.814371,0.677586,0.85,0.781609,0.4,0.590805,8,12,19,68


In [50]:
import numpy as np

from cci.metrics import confusion_matrix

# BUG: Fixed now but..
test_len = 107  # length of test set
for tp, fp, fn, tn in test_df.select(["TP", "FP", "FN", "TN"]).rows():
    tp = test_len - fp - fn - tn
    cm = np.array([[tp, fp], [fn, tn]]).astype("float")
    normalized_cm = cm / cm.sum(axis=1)[:, np.newaxis]
    fig = px.imshow(
        normalized_cm,
        labels=dict(x="Predicted", y="Actual"),
        x=["Good (0)", "Bad (1)"],
        y=["Good (0)", "Bad (1)"],
        text_auto=True,
    )
    fig.show()

In [51]:
def get_df(group) -> pl.DataFrame:
    df = pl.read_csv(res_dir / f"0_{group}.csv").with_columns(
        pl.lit(0).alias("fold"),
        pl.lit(group).alias("group"),
        pl.lit(f"{group}_0").alias("identifier"),
    )

    for i in range(1, 5):
        df = df.vstack(
            pl.read_csv(res_dir / f"{i}_{group}.csv").with_columns(
                pl.lit(i).alias("fold"),
                pl.lit(group).alias("group"),
                pl.lit(f"{group}_{i}").alias("identifier"),
            )
        )

    return df


df = get_df("train")

df = df.vstack(get_df("val"))
df

epoch,acc,loss,f1,auroc,precision,recall,specificity,bac,TP,FP,FN,TN,fold,group,identifier
i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i32,str,str
1,0.542044,0.245985,0.670391,0.5155,0.825688,0.564263,0.437037,0.50065,59,76,278,360,0,"""train""","""train_0"""
2,0.195343,0.243596,0.060423,0.500766,0.833333,0.031348,0.97037,0.500859,131,4,618,20,0,"""train""","""train_0"""
3,0.688228,0.242307,0.807662,0.500296,0.822764,0.793103,0.192593,0.492848,26,109,132,506,0,"""train""","""train_0"""
4,0.760673,0.241967,0.86163,0.522803,0.824034,0.902821,0.088889,0.495855,12,123,62,576,0,"""train""","""train_0"""
5,0.743855,0.242007,0.849544,0.525949,0.824484,0.876176,0.118519,0.497347,16,119,79,559,0,"""train""","""train_0"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
282,0.677725,0.663113,0.779221,0.672381,0.902256,0.685714,0.638889,0.662302,23,13,55,120,4,"""val""","""val_4"""
283,0.672986,0.661076,0.779553,0.669683,0.884058,0.697143,0.555556,0.626349,20,16,53,122,4,"""val""","""val_4"""
284,0.701422,0.620546,0.803738,0.650635,0.883562,0.737143,0.527778,0.63246,19,17,46,129,4,"""val""","""val_4"""


In [52]:
# fig = px.line(df, x="epoch", y="loss", line_group="identifier", color="group", height=800)


def avg_plot(metric: str):
    fig = go.Figure()
    for x, color in (["train", "blue"], ["val", "red"]):
        df = get_df(x)
        loss_df = (
            df.group_by("epoch", maintain_order=True)
            .agg(pl.col(metric).mean().alias("avg"), pl.col(metric).std().alias("std"))
            .to_pandas()
        )
        fig.add_trace(
            go.Scatter(
                name=f"Avg {x} {metric}",
                x=loss_df["epoch"],
                y=loss_df["avg"],
                mode="lines",
            )
        )
        fig.add_trace(
            go.Scatter(
                name=f"{x}upper bound",
                x=loss_df["epoch"],
                y=loss_df["avg"] + loss_df["std"],
                line=dict(width=0),
                marker=dict(color=color),
                showlegend=False,
            )
        )
        fig.add_trace(
            go.Scatter(
                name=f"{x}lower bound",
                x=loss_df["epoch"],
                y=loss_df["avg"] - loss_df["std"],
                line=dict(width=0),
                marker=dict(color=color),
                fill="tonexty",
                showlegend=False,
            ),
        )
    fig.update_layout(hovermode="x", height=800)
    fig.show()


avg_plot("loss")

In [53]:
avg_plot("bac")

In [54]:
avg_plot("acc")

In [55]:
avg_plot("f1")

In [56]:
avg_plot("auroc")